# Odhad CDPF

## Cíl analýzy

Cílem analýzy je odhad neoklasické agrengátní CDPF.

## Data a metoda

Pro odhad byly použity následující hodnoty:

- hrubá přidaná hodnota (Gross Value Added) (převedená na stálé ceny roku 2010 pomocí deflátoru HDP),
- čistý fixní nefinanční kapitál (převedený na stálé ceny roku 2010 pomocí deflátoru kapitálových zásob),
- celkový počet odpracovaných hodin.

In [1]:
import eurostat
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.tsa.stattools import adfuller
from scipy import stats
import statsmodels.tsa.stattools as ts
import numpy
import seaborn
from scipy.stats.mstats import gmean

LOCATIONS = ["CZ", "AT", "EU27_2020"]
BASE_LOCATION = "AT"
START_YEAR = 1996
END_YEAR = 2020

### Hrubá přidaná hodnota (Gross Value Added)

Hrubá přidaná hodnota je načtena na základě kódu `nama_10_a10`.

In [2]:
data_1_es = eurostat.get_data_df('nama_10_a10')

In [3]:
data_1 = data_1_es
# Value added, gross
data_1 = data_1[data_1["na_item"].isin(["B1G", "D1"])]
# Total, all NACE activities
data_1 = data_1[data_1["nace_r2"] == "TOTAL"]
# Current prices, million euro
data_1 = data_1[data_1["unit"].isin(["CP_MNAC", "PD10_NAC"])]
data_1 = pd.melt(data_1, id_vars=["unit", "na_item", "geo\\time"], value_vars=list(range(START_YEAR, END_YEAR + 1)),
                     value_name="value", var_name="year")
data_1 = data_1.pivot(index=["geo\\time", "year"], columns=["unit", "na_item"], values="value")
data_1 = pd.DataFrame(data_1)
data_1.head()

unit            CP_MNAC     PD10_NAC
na_item             B1G  D1      B1G
geo\time year                       
AL       1996  312817.3 NaN      NaN
         1997  304714.2 NaN      NaN
         1998  339154.3 NaN      NaN
         1999  396081.9 NaN      NaN
         2000  442126.7 NaN      NaN

In [4]:
data_1 = data_1.dropna()
data_1[("CP_MNAC", "B1G")] = (data_1[("CP_MNAC", "B1G")] / data_1[("PD10_NAC", "B1G")]) / 100
data_1[("CP_MNAC", "D1")] = (data_1[("CP_MNAC", "D1")] / data_1[("PD10_NAC", "B1G")]) / 100
# data_1 = data_1.drop([("PD10_NAC", "B1G")], axis=1)
data_1.columns = ['_'.join(col) for col in data_1.columns.values]
data_1 = data_1.rename({"CP_MNAC_B1G": "gross_value_added", "CP_MNAC_D1": "compensation_of_employees",
                        "PD10_NAC_B1G": "implicit_deflator_2010"}, axis=1)
data_1 = data_1.reset_index()
data_1 = data_1[data_1["geo\\time"] == "CZ"]
data_1.head()

,geo\time,year,gross_value_added,compensation_of_employees,implicit_deflator_2010
134,CZ,1996,250.814964,111.937994,66.187
135,CZ,1997,247.725862,111.904427,72.489
136,CZ,1998,246.894717,107.043167,79.690
137,CZ,1999,250.422251,107.582155,81.705
138,CZ,2000,261.132320,112.483392,83.245


In [5]:
data_1_gmean = data_1
data_1_gmean["gross_value_added_t-1"] = data_1_gmean.groupby(["geo\\time"])["gross_value_added"].shift()
data_1_gmean = data_1_gmean.dropna()
data_1_gmean["gross_value_added_index"] = data_1_gmean["gross_value_added"] / data_1["gross_value_added_t-1"]
data_1_gmean = data_1_gmean.groupby(["geo\\time"]).agg({"gross_value_added_index": [gmean]}) - 1
data_1_gmean

C:\Users\jirip\AppData\Local\Temp/ipykernel_26196/561914055.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1_gmean["gross_value_added_index"] = data_1_gmean["gross_value_added"] / data_1["gross_value_added_t-1"]


,gross_value_added_index
,gmean
geo\time,
CZ,0.022327


In [6]:
data_1_ln = data_1.sort_values(["geo\\time", "year"])
data_1_ln["ln_gross_value_added"] = numpy.log(data_1_ln["gross_value_added"])
data_1_ln["ln_gross_value_added_t-1"] = data_1_ln.groupby("geo\\time")["ln_gross_value_added"].shift(1)
data_1_ln["ln_gross_value_added_diff"] = data_1_ln["ln_gross_value_added"] - data_1_ln["ln_gross_value_added_t-1"]

data_1_ln["ln_compensation_of_employees"] = numpy.log(data_1_ln["compensation_of_employees"])
data_1_ln["ln_compensation_of_employees_t-1"] = data_1_ln.groupby("geo\\time")["ln_compensation_of_employees"].shift(1)
data_1_ln["ln_compensation_of_employees_diff"] = data_1_ln["ln_compensation_of_employees"] - data_1_ln["ln_compensation_of_employees_t-1"]

data_1_ln = data_1_ln[["geo\\time", "year", "ln_gross_value_added", "ln_gross_value_added_diff", "ln_compensation_of_employees", "ln_compensation_of_employees_diff"]]
data_1_ln.tail()

,geo\time,year,ln_gross_value_added,ln_gross_value_added_diff,ln_compensation_of_employees,ln_compensation_of_employees_diff
154,CZ,2016,5.998628,0.024676,5.231037,0.046560
155,CZ,2017,6.049332,0.050704,5.306536,0.075499
156,CZ,2018,6.082622,0.033290,5.373618,0.067082
157,CZ,2019,6.112236,0.029614,5.407454,0.033837
158,CZ,2020,6.054661,-0.057575,5.373632,-0.033822


In [7]:
data_3_es = eurostat.get_data_df('nama_10_nfa_bs')

In [8]:
data_3 = data_3_es
# Total fixed assets (net)
data_3 = data_3[data_3["asset10"] == "N11N"]
# Total, all activities
data_3 = data_3[data_3["sector"] == "S1"]
# Current fixed prices, millions of national currency
data_3 = data_3[data_3["unit"] == "CP_MNAC"]
data_3 = pd.melt(data_3, id_vars=["unit", "asset10", "geo\\time"], value_vars=list(range(START_YEAR, END_YEAR + 1)),
                    value_name="total_fixed_assets", var_name="year")
data_3 = data_3.sort_values(["geo\\time", "year"])
data_3 = data_3.dropna()
data_3.head()

,unit,asset10,geo\time,year,total_fixed_assets
0,CP_MNAC,N11N,AT,1996,659084.4
29,CP_MNAC,N11N,AT,1997,686219.3
58,CP_MNAC,N11N,AT,1998,710697.9
87,CP_MNAC,N11N,AT,1999,737295.8
116,CP_MNAC,N11N,AT,2000,768843.5


In [9]:
data_4_es = eurostat.get_data_df('nama_10_nfa_fl')

In [10]:
data_4 = data_4_es
data_4 = data_4[data_4["asset10"] == "N11G"]
data_4 = data_4[data_4["nace_r2"] == "TOTAL"]
data_4 = data_4[data_4["unit"] == "PD10_NAC"]
data_4 = pd.melt(data_4, id_vars=["unit", "asset10", "geo\\time"], value_vars=list(range(START_YEAR, END_YEAR + 1)),
                    value_name="capital_stock_deflator", var_name="year")
data_4 = data_4.sort_values(["geo\\time", "year"])
data_4 = data_4.dropna()
data_4.head()


,unit,asset10,geo\time,year,capital_stock_deflator
0,PD10_NAC,N11G,AT,1996,80.723
32,PD10_NAC,N11G,AT,1997,81.648
64,PD10_NAC,N11G,AT,1998,82.293
96,PD10_NAC,N11G,AT,1999,82.710
128,PD10_NAC,N11G,AT,2000,84.036


In [11]:
data_3_merged = data_3.merge(data_4, on=["geo\\time", "year"])
data_3_merged["total_fixed_assets"] = data_3_merged["total_fixed_assets"] / data_3_merged["capital_stock_deflator"]
data_3_merged["total_fixed_assets_t-1"] = data_3_merged.groupby("geo\\time")["total_fixed_assets"].shift(1)
data_3_merged["total_fixed_assets_index"] = data_3_merged["total_fixed_assets"] / data_3_merged["total_fixed_assets_t-1"]
data_3_merged["ln_total_fixed_assets"] = numpy.log(data_3_merged["total_fixed_assets"])
data_3_merged["ln_total_fixed_assets_t-1"] = data_3_merged.groupby("geo\\time")["ln_total_fixed_assets"].shift(1)
data_3_merged["ln_total_fixed_assets_diff"] = data_3_merged["ln_total_fixed_assets"] - data_3_merged["ln_total_fixed_assets_t-1"]
data_3_merged.head()

,unit_x,asset10_x,geo\time,year,total_fixed_assets,unit_y,asset10_y,capital_stock_deflator,total_fixed_assets_t-1,total_fixed_assets_index,ln_total_fixed_assets,ln_total_fixed_assets_t-1,ln_total_fixed_assets_diff
0,CP_MNAC,N11N,AT,1996,8164.765928,PD10_NAC,N11G,80.723,NaN,NaN,9.007583,NaN,NaN
1,CP_MNAC,N11N,AT,1997,8404.606359,PD10_NAC,N11G,81.648,8164.765928,1.029375,9.036535,9.007583,0.028952
2,CP_MNAC,N11N,AT,1998,8636.188983,PD10_NAC,N11G,82.293,8404.606359,1.027554,9.063717,9.036535,0.027181
3,CP_MNAC,N11N,AT,1999,8914.228026,PD10_NAC,N11G,82.710,8636.188983,1.032195,9.095404,9.063717,0.031687
4,CP_MNAC,N11N,AT,2000,9148.977819,PD10_NAC,N11G,84.036,8914.228026,1.026334,9.121397,9.095404,0.025994


In [12]:
data_3_gmean = data_3_merged
data_3_gmean = data_3_gmean.dropna(subset=["total_fixed_assets_index"])
data_3_gmean = data_3_gmean.groupby(["geo\\time"]).agg({"total_fixed_assets_index": [gmean]}) - 1
data_3_gmean

,total_fixed_assets_index
,gmean
geo\time,
AT,0.022338
BE,0.019694
BG,0.020495
CY,0.029168
CZ,0.027041
DE,0.016114
DK,0.019639
EE,0.051596


In [13]:
# Employment by A*10 industry breakdowns
data_5_es = eurostat.get_data_df('nama_10_a10_e')

In [14]:
data_5 = data_5_es

# Value added, gross
data_5 = data_5[data_5["na_item"] == "EMP_DC"]
# Total, all NACE activities
data_5 = data_5[data_5["nace_r2"] == "TOTAL"]
# Thousands hours worked
data_5 = data_5[data_5["unit"] == "THS_HW"]
data_5 = pd.melt(data_5, id_vars=["unit", "na_item", "geo\\time"], value_vars=list(range(START_YEAR, END_YEAR + 1)),
                     value_name="value", var_name="year")
data_5 = data_5.pivot(index=["geo\\time", "year"], columns=["unit", "na_item"], values="value")
data_5 = pd.DataFrame(data_5)
data_5.columns = ['_'.join(col) for col in data_5.columns.values]
data_5 = data_5.rename({"THS_HW_EMP_DC": "thousands_hours_worked"}, axis=1)
data_5 = data_5.reset_index()
data_5.head()

,geo\time,year,thousands_hours_worked
0,AT,1996,6508108.0
1,AT,1997,6586241.0
2,AT,1998,6573561.0
3,AT,1999,6687950.0
4,AT,2000,6753059.0


In [15]:
data_5_ln = data_5.sort_values(["geo\\time", "year"])
data_5_ln["ln_thousands_hours_worked"] = numpy.log(data_5_ln["thousands_hours_worked"])
data_5_ln["ln_thousands_hours_worked_t-1"] = data_5_ln.groupby("geo\\time")["ln_thousands_hours_worked"].shift(1)
data_5_ln["ln_thousands_hours_worked_diff"] = data_5_ln["ln_thousands_hours_worked"] - data_5_ln["ln_thousands_hours_worked_t-1"]

data_5_ln.head()

,geo\time,year,thousands_hours_worked,ln_thousands_hours_worked,ln_thousands_hours_worked_t-1,ln_thousands_hours_worked_diff
0,AT,1996,6508108.0,15.688559,NaN,NaN
1,AT,1997,6586241.0,15.700493,15.688559,0.011934
2,AT,1998,6573561.0,15.698566,15.700493,-0.001927
3,AT,1999,6687950.0,15.715818,15.698566,0.017252
4,AT,2000,6753059.0,15.725506,15.715818,0.009688


In [16]:
data = pd.merge(data_1_ln, data_3_merged, on=["geo\\time", "year"])
data = pd.merge(data, data_5_ln, on=["geo\\time", "year"])
data = data[data["geo\\time"] == "CZ"]
data

,geo\time,year,ln_gross_value_added,ln_gross_value_added_diff,ln_compensation_of_employees,ln_compensation_of_employees_diff,unit_x,asset10_x,total_fixed_assets,unit_y,...,capital_stock_deflator,total_fixed_assets_t-1,total_fixed_assets_index,ln_total_fixed_assets,ln_total_fixed_assets_t-1,ln_total_fixed_assets_diff,thousands_hours_worked,ln_thousands_hours_worked,ln_thousands_hours_worked_t-1,ln_thousands_hours_worked_diff
0,CZ,1996,5.524715,NaN,4.717945,NaN,CP_MNAC,N11N,98722.846780,PD10_NAC,...,77.477,NaN,NaN,11.500072,NaN,NaN,9388353.0,16.054980,NaN,NaN
1,CZ,1997,5.512323,-0.012393,4.717645,-0.000300,CP_MNAC,N11N,104312.104978,PD10_NAC,...,83.065,98722.846780,1.056616,11.555143,11.500072,0.055071,9366006.0,16.052597,16.054980,-0.002383
2,CZ,1998,5.508962,-0.003361,4.673232,-0.044413,CP_MNAC,N11N,109099.885923,PD10_NAC,...,87.660,104312.104978,1.045899,11.600019,11.555143,0.044876,9318501.0,16.047512,16.052597,-0.005085
3,CZ,1999,5.523148,0.014186,4.678255,0.005023,CP_MNAC,N11N,111592.646784,PD10_NAC,...,90.899,109099.885923,1.022848,11.622610,11.600019,0.022591,9263382.0,16.041580,16.047512,-0.005933
4,CZ,2000,5.565027,0.041879,4.722806,0.044551,CP_MNAC,N11N,115473.725575,PD10_NAC,...,92.885,111592.646784,1.034779,11.656798,11.622610,0.034188,9234010.0,16.038404,16.041580,-0.003176
5,CZ,2001,5.595260,0.030233,4.751364,0.028558,CP_MNAC,N11N,119721.362916,PD10_NAC,...,94.650,115473.725575,1.036784,11.692922,11.656798,0.036124,8836536.0,15.994406,16.038404,-0.043998
6,CZ,2002,5.615320,0.020060,4.797132,0.045768,CP_MNAC,N11N,126103.076558,PD10_NAC,...,92.766,119721.362916,1.053305,11.744855,11.692922,0.051933,8837763.0,15.994544,15.994406,0.000139
7,CZ,2003,5.644631,0.029311,4.832204,0.035072,CP_MNAC,N11N,129446.456726,PD10_NAC,...,93.868,126103.076558,1.026513,11.771023,11.744855,0.026168,8703898.0,15.979282,15.994544,-0.015263
8,CZ,2004,5.692031,0.047399,4.883533,0.051329,CP_MNAC,N11N,132888.830074,PD10_NAC,...,96.348,129446.456726,1.026593,11.797268,11.771023,0.026246,8757937.0,15.985471,15.979282,0.006189
9,CZ,2005,5.757599,0.065569,4.954332,0.070799,CP_MNAC,N11N,137703.005814,PD10_NAC,...,97.012,132888.830074,1.036227,11.832855,11.797268,0.035586,8875156.0,15.998766,15.985471,0.013296


In [17]:
data = data.dropna()
data = data[["ln_gross_value_added_diff", "ln_thousands_hours_worked_diff", "ln_total_fixed_assets_diff", "geo\\time", "year"]]
data

,ln_gross_value_added_diff,ln_thousands_hours_worked_diff,ln_total_fixed_assets_diff,geo\time,year
1,-0.012393,-0.002383,0.055071,CZ,1997
2,-0.003361,-0.005085,0.044876,CZ,1998
3,0.014186,-0.005933,0.022591,CZ,1999
4,0.041879,-0.003176,0.034188,CZ,2000
5,0.030233,-0.043998,0.036124,CZ,2001
6,0.020060,0.000139,0.051933,CZ,2002
7,0.029311,-0.015263,0.026168,CZ,2003
8,0.047399,0.006189,0.026246,CZ,2004
9,0.065569,0.013296,0.035586,CZ,2005
10,0.070281,0.005760,0.044329,CZ,2006


In [18]:
# Employment by A*10 industry breakdowns
data_6_es = eurostat.get_data_df('nama_10_lp_ulc')

In [19]:
data_6 = data_6_es
# Compensation of employees per hour worked
data_6 = data_6[data_6["na_item"] == "D1_SAL_HW"]
# # Current prices, million euro
data_6 = data_6[data_6["unit"].isin(["NAC"])]
data_6 = pd.melt(data_6, id_vars=["unit", "na_item", "geo\\time"], value_vars=list(range(START_YEAR, END_YEAR + 1)),
                     value_name="value", var_name="year")
data_6 = data_6.pivot(index=["geo\\time", "year"], columns=["unit", "na_item"], values="value")
data_6.columns = ['_'.join(col) for col in data_6.columns.values]
data_6 = data_6.rename({"NAC_D1_SAL_HW": "compensation_per_hour"}, axis=1)
data_6 = data_6.reset_index()
data_6.head()

,geo\time,year,compensation_per_hour
0,AT,1996,17.7
1,AT,1997,17.8
2,AT,1998,18.5
3,AT,1999,18.8
4,AT,2000,19.3


In [20]:
data["const"] = 1

import statsmodels.api as sm
mod = sm.OLS(data["ln_gross_value_added_diff"], data[["const", "ln_thousands_hours_worked_diff", "ln_total_fixed_assets_diff"]])
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                OLS Regression Results                               
=====================================================================================
Dep. Variable:     ln_gross_value_added_diff   R-squared:                       0.461
Model:                                   OLS   Adj. R-squared:                  0.409
Method:                        Least Squares   F-statistic:                     8.965
Date:                       Sun, 23 Jan 2022   Prob (F-statistic):            0.00153
Time:                               17:37:46   Log-Likelihood:                 56.423
No. Observations:                         24   AIC:                            -106.8
Df Residuals:                             21   BIC:                            -103.3
Df Model:                                  2                                         
Covariance Type:                   nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                              0.0142      0.008      1.718      0.101      -0.003       0.031
ln_thousands_hours_worked_diff     0.8993      0.256      3.515      0.002       0.367       1.431
ln_total_fixed_assets_diff         0.3411      0.241      1.414      0.172      -0.161       0.843
==============================================================================
Omnibus:                        0.442   Durbin-Watson:                   1.366
Prob(Omnibus):                  0.802   Jarque-Bera (JB):                0.563
Skew:                          -0.118   Prob(JB):                        0.755
Kurtosis:                       2.288   Cond. No.                         55.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [21]:
data["resid"] = res.resid
data["ln_gross_value_added_diff+1"] = data["ln_gross_value_added_diff"] + 1
gmean(data["ln_gross_value_added_diff+1"]) - 1

0.021588139671504525

In [22]:
res.resid

1    -0.043194
2    -0.028254
3    -0.002343
4     0.018914
5     0.043321
6    -0.011939
7     0.019953
8     0.018722
9     0.027314
10    0.035821
11    0.001430
12   -0.028388
13   -0.043509
14    0.009940
15   -0.005218
16   -0.006037
17   -0.010748
18    0.002779
19    0.027003
20   -0.020661
21    0.017111
22   -0.006754
23    0.005301
24   -0.020565
dtype: float64

In [23]:
data["TFP3"] = data["resid"] + res.params[0]
data

,ln_gross_value_added_diff,ln_thousands_hours_worked_diff,ln_total_fixed_assets_diff,geo\time,year,const,resid,ln_gross_value_added_diff+1,TFP3
1,-0.012393,-0.002383,0.055071,CZ,1997,1,-0.043194,0.987607,-0.029035
2,-0.003361,-0.005085,0.044876,CZ,1998,1,-0.028254,0.996639,-0.014096
3,0.014186,-0.005933,0.022591,CZ,1999,1,-0.002343,1.014186,0.011816
4,0.041879,-0.003176,0.034188,CZ,2000,1,0.018914,1.041879,0.033073
5,0.030233,-0.043998,0.036124,CZ,2001,1,0.043321,1.030233,0.057480
6,0.020060,0.000139,0.051933,CZ,2002,1,-0.011939,1.020060,0.002220
7,0.029311,-0.015263,0.026168,CZ,2003,1,0.019953,1.029311,0.034111
8,0.047399,0.006189,0.026246,CZ,2004,1,0.018722,1.047399,0.032880
9,0.065569,0.013296,0.035586,CZ,2005,1,0.027314,1.065569,0.041473
10,0.070281,0.005760,0.044329,CZ,2006,1,0.035821,1.070281,0.049980


In [24]:
data["TFP3+1"] = data["TFP3"] + 1
gmean(data["TFP3+1"]) - 1

0.013895987342491312

In [25]:
data["%"] = data["TFP3"] / data["ln_gross_value_added_diff"]
data["%"].mean()


-0.18048708821006762

In [26]:
data_final_2 = data_1[["geo\\time", "year", "gross_value_added", "implicit_deflator_2010"]]
data_final_2 = data_final_2.merge(data_6, on=["geo\\time", "year"])
data_final_2 = data_final_2.merge(data_5_ln, on=["geo\\time", "year"])
data_final_2["total_compensations"] = (data_final_2["thousands_hours_worked"] * data_final_2["compensation_per_hour"])
data_final_2["total_compensations"] = (data_final_2["total_compensations"] / data_final_2["implicit_deflator_2010"]) / 100
data_final_2["total_compensations"] = data_final_2["total_compensations"] / 10 ** 3
data_final_2["1 - alpha"] = data_final_2["total_compensations"] / data_final_2["gross_value_added"]
# data_final_2 = data_final_2[data_final_2["geo\\time"] == "CZ"]
# data_final_2["alpha"] = 1 - data_final_2["1-alpha"]
data_final_2


,geo\time,year,gross_value_added,implicit_deflator_2010,compensation_per_hour,thousands_hours_worked,ln_thousands_hours_worked,ln_thousands_hours_worked_t-1,ln_thousands_hours_worked_diff,total_compensations,1 - alpha
0,CZ,1996,250.814964,66.187,93.9,9388353.0,16.054980,NaN,NaN,133.193278,0.531042
1,CZ,1997,247.725862,72.489,103.3,9366006.0,16.052597,16.054980,-0.002383,133.469688,0.538780
2,CZ,1998,246.894717,79.690,111.4,9318501.0,16.047512,16.052597,-0.005085,130.264903,0.527613
3,CZ,1999,250.422251,81.705,116.7,9263382.0,16.041580,16.047512,-0.005933,132.309734,0.528347
4,CZ,2000,261.132320,83.245,125.6,9234010.0,16.038404,16.041580,-0.003176,139.322681,0.533533
5,CZ,2001,269.147676,87.455,142.7,8836536.0,15.994406,16.038404,-0.043998,144.185431,0.535711
6,CZ,2002,274.601272,89.610,154.7,8837763.0,15.994544,15.994406,0.000139,152.572474,0.555615
7,CZ,2003,282.769251,91.281,168.8,8703898.0,15.979282,15.994544,-0.015263,160.955509,0.569211
8,CZ,2004,296.495055,94.030,179.7,8757937.0,15.985471,15.979282,0.006189,167.372251,0.564503
9,CZ,2005,316.587437,93.862,185.9,8875156.0,15.998766,15.985471,0.013296,175.778430,0.555229
